In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns

In [32]:
#Pull in base dataframe and add avg departure delay
df = pd.read_csv('../CleaningTableFlights/data/flights_test.csv')
dep_delay = pd.read_csv('../CleaningTableFlights/data/flights.csv')[['dep_delay','origin']]
arr_delay = pd.read_csv('../CleaningTableFlights/data/flights.csv')[['arr_delay','dest']]

In [33]:
#departure delay
dep_delay = dep_delay.groupby('origin').mean()
dep_delay = dep_delay.rename({'dep_delay':'average_departure_delay_by_ap'},axis=1)

In [34]:
df = df.drop(columns=['branded_code_share','mkt_carrier','op_carrier_fl_num','origin_airport_id','origin_city_name','dest_airport_id','dest_city_name','dup','flights'])

In [35]:
#add constant
df['constant'] = 1
first_column = df.pop('constant')
df.insert(0, 'constant', first_column)

In [36]:
#arr_delay data by airport
arr_delay = arr_delay.groupby('dest').mean()
arr_delay = arr_delay.rename({'arr_delay':'average_arrival_delay_by_ap_full'},axis=1)

In [37]:
arr_delay.head()

,average_arrival_delay_by_ap_full
dest,
ABE,6.937888
ABI,-1.176471
ABQ,6.361068
ABR,0.818182
ABY,6.000000


In [38]:
#passenger_fuel_info by carrier
pass_fuel_df = pd.read_csv('data/passenger_fuel_df_full.csv')
pass_fuel_df.head()

,mkt_unique_carrier,avg_distance_per_month_by_carrier,avg_dep_delay_by_carrier,total_gallons,passengers_by_carrier_per_month,monthly_distance_per_passenger,avgfuel_percustomer_perdistance
0,AA,1.198062e+08,1.732355e+06,529073.835069,1.269777e+07,0.105986,4.991926e+06
1,AS,3.493391e+07,1.290336e+05,121682.671875,2.920384e+06,0.083597,1.455580e+06
2,B6,2.733835e+07,4.146305e+05,147726.593750,3.545438e+06,0.129687,1.139098e+06
3,DL,1.014802e+08,1.347398e+06,547151.206597,1.313163e+07,0.129401,4.228340e+06
4,F9,1.099285e+07,1.775858e+05,74017.159722,1.776412e+06,0.161597,4.580353e+05


In [39]:
#departures info by year per airport
departures_df = pd.read_csv('data/departures_by_airport_full.csv')
departures_df = departures_df.drop('origin_city_name',axis=1)
departures_df.head()

,origin,yearly_departures_per_ap,yearly_passengers_per_ap
0,05A,75.4,149.6
1,06A,7.2,3.8
2,08A,2.0,3.0
3,09A,7.2,8.6
4,1B1,1.8,5.8


In [40]:
#arrival info by year per airport
arrivals_df = pd.read_csv('data/arrivals_by_airport.csv')
arrivals_df = arrivals_df.drop('dest_city_name',axis=1)
arrivals_df.head()

,dest,yearly_arrivals_per_ap,yearly_arriving_passengers_per_ap
0,05A,76.6,168.6
1,06A,7.0,3.6
2,08A,2.2,3.4
3,09A,7.4,13.0
4,1B1,1.2,3.2


In [41]:
#monthly destination and origin info
month_dest_df = pd.read_csv('data/features_dest_monthly_params.csv')
month_origin_df = pd.read_csv('data/features_origin_monthly_params.csv')

In [42]:
#create month and weekday variables
df['month'] = pd.DatetimeIndex(df['fl_date']).month
df['weekday'] = pd.DatetimeIndex(df['fl_date']).weekday

In [43]:
#create combo variables for grouping
month_dest_df['combo1']=month_dest_df['dest'].astype('str')+month_dest_df['fl_month'].astype('str')
df['combo1']=df['dest'].astype('str')+df['month'].astype('str')

#rename columns for clarity and drop useless columns
month_dest_df = month_dest_df.drop(['fl_month','dest'],axis=1)
month_dest_df = month_dest_df.rename(columns={'nas_delay':'avg_nas_delay_by_month_by_dest_ap','carrier_delay':'avg_carrier_delay_by_month_by_dest_ap','arr_delay':'avg_arr_delay_by_month_by_dest_ap','cancelled':'cancelled_flights_by_month_by_dest_ap','weather_delay':'avg_weather_delay_by_month_by_dest_ap','security_delay':'security_delay_by_month_by_dest_ap'})
month_dest_df.head()

,avg_arr_delay_by_month_by_dest_ap,avg_carrier_delay_by_month_by_dest_ap,avg_nas_delay_by_month_by_dest_ap,late_aircraft_delay,avg_weather_delay_by_month_by_dest_ap,security_delay_by_month_by_dest_ap,cancelled_flights_by_month_by_dest_ap,combo1
0,6.361596,4.544888,2.058603,5.751870,1.372818,0.004988,0.024938,ABE1
1,7.712614,6.074122,1.954486,6.189857,1.872562,0.000000,0.027308,ABE2
2,3.069915,3.960805,2.456568,3.204449,0.309322,0.000000,0.040254,ABE3
3,2.710451,2.215724,1.658677,4.325983,1.352828,0.007670,0.019175,ABE4
4,3.248503,3.714571,1.660679,3.884232,0.717565,0.003992,0.016966,ABE5


In [44]:
#create combo variables for grouping
month_origin_df['combo2']=month_origin_df['origin'].astype('str')+month_origin_df['fl_month'].astype('str')
df['combo2']=df['origin'].astype('str')+df['month'].astype('str')

#rename columns for clarity and drop useless columns
month_origin_df = month_origin_df.drop(['fl_month','origin'],axis=1)
month_origin_df = month_origin_df.rename(columns={'dep_delay':'avg_dep_delay_by_month_by_origin_ap','nas_delay':'avg_nas_delay_by_month_by_origin_ap','carrier_delay':'avg_carrier_delay_by_month_by_origin_ap','arr_delay':'avg_arr_delay_by_month_by_origin_ap','cancelled':'cancelled_flights_by_month_by_origin_ap','weather_delay':'avg_weather_delay_by_month_by_origin_ap','security_delay':'security_delay_by_month_by_origin_ap'})
month_origin_df.head()

,avg_dep_delay_by_month_by_origin_ap,avg_arr_delay_by_month_by_origin_ap,avg_carrier_delay_by_month_by_origin_ap,avg_nas_delay_by_month_by_origin_ap,late_aircraft_delay,avg_weather_delay_by_month_by_origin_ap,security_delay_by_month_by_origin_ap,cancelled_flights_by_month_by_origin_ap,combo2
0,12.186250,8.980000,6.170000,4.212500,5.995000,0.497500,0.002500,0.026250,ABE1
1,15.410390,14.646753,5.358442,4.472727,8.574026,2.341558,0.000000,0.031169,ABE2
2,7.479873,5.648305,3.481992,4.149364,5.019068,0.307203,0.020127,0.042373,ABE3
3,9.223823,7.659942,5.782901,2.552354,4.845341,0.872238,0.013449,0.015370,ABE4
4,8.562874,3.111776,2.921158,2.324351,6.011976,0.756487,0.000000,0.015968,ABE5


In [45]:
#daily arrival info - unused
daily_arrivals = pd.read_csv('data/num_arrivals_daily.csv')
daily_departures = pd.read_csv('data/num_departures_daily.csv')

In [46]:
daily_arrivals.head()

,fl_date,dest,num_flights_arriving
0,2018-01-01,ABE,10
1,2018-01-01,ABI,6
2,2018-01-01,ABQ,77
3,2018-01-01,ABR,2
4,2018-01-01,ABY,2


In [47]:
#tailnumber info - unused
tailnum_df = pd.read_csv('data/features_tailnum_delay_taxi_median_params.csv')
tailnum_df = tailnum_df.rename(columns={'dep_delay':'dep_delay_by_plane','taxi_out':'taxi_out_by_plane','arr_delay':'arr_delay_by_plane','taxi_in':'taxi_in_by_plane'})

In [48]:
tailnum_df.head()

,tail_num,dep_delay_by_plane,taxi_out_by_plane,arr_delay_by_plane,taxi_in_by_plane
0,215NV,-1.0,11.0,-4.0,6.0
1,216NV,-3.0,12.0,-5.0,6.0
2,217NV,-2.0,11.0,-3.0,7.0
3,218NV,-2.0,11.0,-4.0,6.0
4,219NV,-4.0,11.0,-6.0,6.0


In [49]:
#convert time to # of minute since midnight
df['crs_arr_time'] = (df['crs_arr_time']//100)*60 + df['crs_arr_time'] - (df['crs_arr_time']//100)*100
df['crs_dep_time'] = (df['crs_dep_time']//100)*60 + df['crs_dep_time'] - (df['crs_dep_time']//100)*100

In [50]:
#add in fuel and passenger info by carrier
df = df.merge(pass_fuel_df, how = 'left', left_on='mkt_unique_carrier',right_on='mkt_unique_carrier')

In [51]:
#add in departure and passenger info by airport
df = df.merge(departures_df, how = 'left', on = 'origin')

#add in arrival and passenger info by arrival airport
df = df.merge(arrivals_df, how = 'left', on = 'dest')

#add in average departure delay by departure location
df = df.merge(dep_delay, how = 'left', left_on = 'origin', right_index = True)

#add in average arrival delay by arrival location
df = df.merge(arr_delay, how = 'left', left_on = 'origin', right_index = True)

#add in monthly data
df = df.merge(month_dest_df, how = 'left', on = 'combo1')
df = df.merge(month_origin_df, how = 'left', on = 'combo2')

#Below excluded because their addition INCREASED RMSE
# #add in departures & arrivals per day
# df = df.merge(daily_arrivals, how = 'left', on = ['fl_date','dest'])
# df = df.merge(daily_departures, how = 'left', on = ['fl_date','origin'])

# #add in plane level info
# df = df.merge(tailnum_df, how='left', on = 'tail_num')

In [52]:
df = df.drop(['fl_date','tail_num','mkt_carrier_fl_num'], axis = 1)

In [53]:
#create variable where 1 if op_carrier different from mkt_carrier
df['op_unique_carrier_check'] = df['op_unique_carrier'] != df['mkt_unique_carrier']
df['op_unique_carrier_check'] = df['op_unique_carrier_check'].astype('int')

In [54]:
#fill arrival/departure average nans with 0
df = df.fillna(0)

In [56]:
df = df.drop(['origin','dest','op_unique_carrier','combo1','combo2'],axis=1)

In [57]:
df.columns

Index(['constant', 'mkt_unique_carrier', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance', 'month', 'weekday',
       'avg_distance_per_month_by_carrier', 'avg_dep_delay_by_carrier',
       'total_gallons', 'passengers_by_carrier_per_month',
       'monthly_distance_per_passenger', 'avgfuel_percustomer_perdistance',
       'yearly_departures_per_ap', 'yearly_passengers_per_ap',
       'yearly_arrivals_per_ap', 'yearly_arriving_passengers_per_ap',
       'average_departure_delay_by_ap', 'average_arrival_delay_by_ap_full',
       'avg_arr_delay_by_month_by_dest_ap',
       'avg_carrier_delay_by_month_by_dest_ap',
       'avg_nas_delay_by_month_by_dest_ap', 'late_aircraft_delay_x',
       'avg_weather_delay_by_month_by_dest_ap',
       'security_delay_by_month_by_dest_ap',
       'cancelled_flights_by_month_by_dest_ap',
       'avg_dep_delay_by_month_by_origin_ap',
       'avg_arr_delay_by_month_by_origin_ap',
       'avg_carrier_delay_by_month_by_origin_ap',
       '

In [58]:
df.head()

,constant,mkt_unique_carrier,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,month,weekday,avg_distance_per_month_by_carrier,avg_dep_delay_by_carrier,...,cancelled_flights_by_month_by_dest_ap,avg_dep_delay_by_month_by_origin_ap,avg_arr_delay_by_month_by_origin_ap,avg_carrier_delay_by_month_by_origin_ap,avg_nas_delay_by_month_by_origin_ap,late_aircraft_delay_y,avg_weather_delay_by_month_by_origin_ap,security_delay_by_month_by_origin_ap,cancelled_flights_by_month_by_origin_ap,op_unique_carrier_check
0,1,WN,1090,1185,95,363,1,2,84282361.5,1.183564e+06,...,0.020789,5.854884,0.125635,1.62987,2.872106,3.360248,0.061547,0.037267,0.008187,0
1,1,WN,710,800,90,363,1,2,84282361.5,1.183564e+06,...,0.020789,5.854884,0.125635,1.62987,2.872106,3.360248,0.061547,0.037267,0.008187,0
2,1,WN,1220,1290,70,333,1,2,84282361.5,1.183564e+06,...,0.006971,5.854884,0.125635,1.62987,2.872106,3.360248,0.061547,0.037267,0.008187,0
3,1,WN,820,895,75,333,1,2,84282361.5,1.183564e+06,...,0.006971,5.854884,0.125635,1.62987,2.872106,3.360248,0.061547,0.037267,0.008187,0
4,1,WN,555,635,80,333,1,2,84282361.5,1.183564e+06,...,0.006971,5.854884,0.125635,1.62987,2.872106,3.360248,0.061547,0.037267,0.008187,0


In [59]:
dummy_df = pd.get_dummies(df,columns=['mkt_unique_carrier','month','weekday'])

In [61]:
#drop final category of each dummy variable
dummy_df = dummy_df.drop(columns=['weekday_6','mkt_unique_carrier_WN'])

In [70]:
dummy_df.columns

Index(['constant', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time',
       'distance', 'avg_distance_per_month_by_carrier',
       'avg_dep_delay_by_carrier', 'total_gallons',
       'passengers_by_carrier_per_month', 'monthly_distance_per_passenger',
       'avgfuel_percustomer_perdistance', 'yearly_departures_per_ap',
       'yearly_passengers_per_ap', 'yearly_arrivals_per_ap',
       'yearly_arriving_passengers_per_ap', 'average_departure_delay_by_ap',
       'average_arrival_delay_by_ap_full', 'avg_arr_delay_by_month_by_dest_ap',
       'avg_carrier_delay_by_month_by_dest_ap',
       'avg_nas_delay_by_month_by_dest_ap', 'late_aircraft_delay_x',
       'avg_weather_delay_by_month_by_dest_ap',
       'security_delay_by_month_by_dest_ap',
       'cancelled_flights_by_month_by_dest_ap',
       'avg_dep_delay_by_month_by_origin_ap',
       'avg_arr_delay_by_month_by_origin_ap',
       'avg_carrier_delay_by_month_by_origin_ap',
       'avg_nas_delay_by_month_by_origin_ap', 'late_aircr

In [76]:
dummy_df

,constant,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,avg_distance_per_month_by_carrier,avg_dep_delay_by_carrier,total_gallons,passengers_by_carrier_per_month,monthly_distance_per_passenger,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,mkt_unique_carrier_VX
0,1,1090,1185,95,363,8.428236e+07,1.183564e+06,581150.196181,1.394760e+07,0.165487,...,0,0,0,0,0,0,0,0,0,0
1,1,710,800,90,363,8.428236e+07,1.183564e+06,581150.196181,1.394760e+07,0.165487,...,0,0,0,0,0,0,0,0,0,0
2,1,1220,1290,70,333,8.428236e+07,1.183564e+06,581150.196181,1.394760e+07,0.165487,...,0,0,0,0,0,0,0,0,0,0
3,1,820,895,75,333,8.428236e+07,1.183564e+06,581150.196181,1.394760e+07,0.165487,...,0,0,0,0,0,0,0,0,0,0
4,1,555,635,80,333,8.428236e+07,1.183564e+06,581150.196181,1.394760e+07,0.165487,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660857,1,1139,1241,102,411,1.014802e+08,1.347398e+06,547151.206597,1.313163e+07,0.129401,...,0,0,0,0,0,0,0,0,0,0
660858,1,915,1022,107,411,1.014802e+08,1.347398e+06,547151.206597,1.313163e+07,0.129401,...,0,0,0,0,0,0,0,0,0,0
660859,1,1325,1417,92,266,1.014802e+08,1.347398e+06,547151.206597,1.313163e+07,0.129401,...,0,0,0,0,0,0,0,0,0,0
660860,1,635,836,141,740,1.014802e+08,1.347398e+06,547151.206597,1.313163e+07,0.129401,...,0,0,0,0,0,0,0,0,0,0


In [79]:
#adding dummies for variables not present
dummy_df['month_2']=0
dummy_df['month_3']=0
dummy_df['month_4']=0
dummy_df['month_5']=0
dummy_df['month_6']=0
dummy_df['month_7']=0
dummy_df['month_8']=0
dummy_df['month_9']=0
dummy_df['month_10']=0
dummy_df['month_11']=0
dummy_df['mkt_unique_carrier_VX']=0

#reorder
dummy_df = dummy_df[['constant', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time',
       'distance', 'avg_distance_per_month_by_carrier',
       'avg_dep_delay_by_carrier', 'total_gallons',
       'passengers_by_carrier_per_month', 'monthly_distance_per_passenger',
       'avgfuel_percustomer_perdistance', 'yearly_departures_per_ap',
       'yearly_passengers_per_ap', 'yearly_arrivals_per_ap',
       'yearly_arriving_passengers_per_ap', 'average_departure_delay_by_ap',
       'average_arrival_delay_by_ap_full', 'avg_arr_delay_by_month_by_dest_ap',
       'avg_carrier_delay_by_month_by_dest_ap',
       'avg_nas_delay_by_month_by_dest_ap', 'late_aircraft_delay_x',
       'avg_weather_delay_by_month_by_dest_ap',
       'security_delay_by_month_by_dest_ap',
       'cancelled_flights_by_month_by_dest_ap',
       'avg_dep_delay_by_month_by_origin_ap',
       'avg_arr_delay_by_month_by_origin_ap',
       'avg_carrier_delay_by_month_by_origin_ap',
       'avg_nas_delay_by_month_by_origin_ap', 'late_aircraft_delay_y',
       'avg_weather_delay_by_month_by_origin_ap',
       'security_delay_by_month_by_origin_ap',
       'cancelled_flights_by_month_by_origin_ap', 'op_unique_carrier_check',
       'mkt_unique_carrier_AA', 'mkt_unique_carrier_AS',
       'mkt_unique_carrier_B6', 'mkt_unique_carrier_DL',
       'mkt_unique_carrier_F9', 'mkt_unique_carrier_G4',
       'mkt_unique_carrier_HA', 'mkt_unique_carrier_NK',
       'mkt_unique_carrier_UA', 'mkt_unique_carrier_VX', 'month_1', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'weekday_0', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5']]

XGBoost Regression Model

In [72]:
import pickle as pkl

In [81]:
filenm = 'XGBoostModel.pickle'
model_from_pickle = pkl.load(open(filenm,'rb'))
y_pred = model_from_pickle.predict(dummy_df)

In [90]:
export_df = pd.read_csv('../CleaningTableFlights/data/flights_test.csv')

export_df = export_df[['fl_date','mkt_carrier','mkt_carrier_fl_num','origin','dest']]

df['predicted_delay']=y_pred

export_df = export_df.merge(df['predicted_delay'],how='left',left_index = True, right_index=True)

In [92]:
export_df.t

fl_date               0
mkt_carrier           0
mkt_carrier_fl_num    0
origin                0
dest                  0
predicted_delay       0
dtype: int64